In [1]:
import pandas as pd
import snowflake.connector
import os
from dotenv import load_dotenv

from process_data import get_trains_data, get_events


get_trains_data("AMB")

/Users/henrieloy/ideo-POC/venv/lib/python3.11/site-packages/snowflake/connector/options.py:104: UserWarning: You have an incompatible version of 'pyarrow' installed (20.0.0), please install a version that adheres to: 'pyarrow<19.0.0; extra == "pandas"'
  warn_incompatible_dep(
/Users/henrieloy/ideo-POC/process_data.py:185: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,TRAIN_ID,DEPARTURE_POINT,ARRIVAL_POINT,TYPE,DEPARTURE_DATE,ARRIVAL_DATE,NB_WAGONS
0,D_EV_AMB_202529749,EV,AMB,Evac,2025-05-25 21:37:00,2025-05-25 23:59:00,21
1,D_AMB_EV_202529725,AMB,EV,Appro,2025-05-25 20:48:00,2025-05-25 23:11:00,28
2,D_AMB_EV_202529726,AMB,EV,Appro,2025-05-25 17:38:00,2025-05-25 20:16:00,28
3,D_AMB_DOU_202529661,AMB,DOU,Chargés,2025-05-25 09:14:00,2025-05-26 10:14:00,30
4,D_AMB_RUG_202529674,AMB,RUG,Chargés,2025-05-25 08:07:00,2025-05-27 20:15:00,22
...,...,...,...,...,...,...,...
173,D_EV_AMB_202529550,EV,AMB,Evac,2025-05-12 12:38:00,2025-05-12 12:38:00,12
174,D_AMB_EV_202529530,AMB,EV,Appro,2025-05-12 10:18:00,2025-05-12 13:24:00,13
175,D_AMB_ANV_202529504,AMB,ANV,Chargés,2025-05-12 08:46:00,2025-05-15 07:04:00,30
176,D_EV_AMB_202529549,EV,AMB,Evac,2025-05-12 06:10:00,2025-05-12 08:53:00,6


In [2]:
def compute_stocks(location=None):
    """Calcule les stocks de wagonspour une période donnée et une localisation donnée"""

    # Récupérer les données des trains
    trains_data = get_trains_data(location)
    
    # Créer une liste pour stocker tous les événements (arrivées et départs)
    events = []


    for _, row in trains_data.iterrows():
        departure_date = row['DEPARTURE_DATE']
        arrival_date = row['ARRIVAL_DATE']
        nb_wagons = row['NB_WAGONS']

        if pd.notna(departure_date):
            to_append = {
                'datetime': departure_date,
                'location': row['DEPARTURE_POINT'],
                'train_id': row['TRAIN_ID'],
                'event_type': 'departure',
                'change': -nb_wagons,  # Le train quitte ce lieu
            }
            if location == "AMB":
                to_append['status'] = "plein" if row['TYPE'] == "Chargés" else "vide"
            events.append(to_append)


        if pd.notna(arrival_date):
            to_append = {
                'datetime': arrival_date,
                'location': row['ARRIVAL_POINT'],
                'train_id': row['TRAIN_ID'],
                'event_type': 'arrival',
                'change': nb_wagons,  # Le train arrive dans ce lieu
            }
            if location == "AMB":
                to_append['status'] = "plein" if row['TYPE'] == "Evac" else "vide"
            events.append(to_append)

    # Convertir en DataFrame et trier par datetime
    events_df = pd.DataFrame(events)
    
    # Supprimer les doublons exacts (même train, même lieu, même datetime, même type)
    if location == "AMB":
        events_df = events_df.drop_duplicates(subset=['datetime', 'location', 'train_id', 'event_type', 'status']).reset_index(drop=True)
    else:
        events_df = events_df.drop_duplicates(subset=['datetime', 'location', 'train_id', 'event_type']).reset_index(drop=True)
    
    events_df = events_df.sort_values(['location', 'datetime']).reset_index(drop=True)

    # Calculer le nombre cumulé de trains par lieu et datetime
    if location == "AMB":
        events_df['cumulative_wagons'] = events_df.groupby(['location', 'status'])['change'].cumsum()
    else:
        events_df['cumulative_wagons'] = events_df.groupby('location')['change'].cumsum()

    # Créer le dataframe final avec le nombre de trains à chaque moment et lieu
    if location == "AMB":
        train_count_df = events_df[['datetime', 'location', 'status', 'cumulative_wagons']].copy()
        train_count_df = train_count_df.rename(columns={'cumulative_wagons': 'nombre_wagons'})
    else:
        train_count_df = events_df[['datetime', 'location', 'cumulative_wagons']].copy()
        train_count_df = train_count_df.rename(columns={'cumulative_wagons': 'nombre_wagons'})

    if location:
        train_count_df = train_count_df[train_count_df['location'] == location]
    
    return train_count_df

In [3]:
compute_stocks("WOR")

/Users/henrieloy/ideo-POC/process_data.py:185: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,datetime,location,nombre_wagons
8,2025-05-16 08:02:00,WOR,28
9,2025-05-20 09:30:00,WOR,56
10,2025-05-21 08:01:00,WOR,84
11,2025-05-22 08:00:00,WOR,112
12,2025-05-23 08:04:00,WOR,140
13,2025-05-23 22:29:00,WOR,168
14,2025-05-27 10:30:00,WOR,196
15,2025-05-28 08:00:00,WOR,224


In [ ]:
def apply_corrections(location=None):
    """Applique les corrections aux stocks"""
    # Récupérer les événements de correction
    corrections = get_events(location)
    wagons_count_df = compute_stocks(location)
    
    if corrections.empty:
        return wagons_count_df
    
    # Créer une copie du dataframe pour éviter de modifier l'original
    wagons_count_df1 = wagons_count_df.copy()
    
    # Trier les corrections par date
    corrections = corrections.sort_values('EVENT_DATE').reset_index(drop=True)
    
    # Pour chaque lieu dans les données de wagons
    for loc in wagons_count_df1['location'].unique():
        # Filtrer les corrections pour ce lieu
        loc_corrections = corrections[corrections['LOCATION'] == loc]
        
        if loc_corrections.empty:
            continue
            
        # Filtrer les données de wagons pour ce lieu
        loc_mask = wagons_count_df1['location'] == loc
        loc_data = wagons_count_df1[loc_mask].copy()

        
        # Appliquer les corrections chronologiquement
        for _, correction in loc_corrections.iterrows():
            correction_date = correction['EVENT_DATE']
            nb_wagons = correction['NB_WAGONS']
            is_relative = correction['RELATIVE']

            past_mask = loc_data['datetime'] < correction_date


            # Créer une nouvelle ligne avec la correction
            nouvelle_ligne = pd.DataFrame({
                'datetime': [correction_date],
                'location': [loc],
                'nombre_wagons': loc_data[past_mask]['nombre_wagons'].iloc[-1]
            })
            wagons_count_df1 = pd.concat([wagons_count_df1, nouvelle_ligne], ignore_index=True)

            # Trouver les indices des lignes à partir de cette date
            future_mask = loc_data['datetime'] >= correction_date


            
            if is_relative:
                # Correction relative : ajouter/soustraire le nombre de wagons
                loc_data.loc[future_mask, 'nombre_wagons'] += nb_wagons

            else:
                # Correction absolue : calculer la différence et l'appliquer
                # Trouver la valeur juste avant la correction
                if past_mask.any():
                    # Prendre la dernière valeur avant la correction
                    last_value = loc_data[past_mask]['nombre_wagons'].iloc[-1]
                else:
                    # Si pas de valeur précédente, commencer à 0
                    last_value = 0
                
                # Calculer la différence à appliquer
                target_value = nb_wagons
                difference = target_value - last_value
                
                # Appliquer la différence à partir de cette date
                loc_data.loc[future_mask, 'nombre_wagons'] += difference
        
        # Mettre à jour le dataframe principal
        wagons_count_df1.loc[loc_mask, 'nombre_wagons'] = loc_data['nombre_wagons']
    
    return wagons_count_df1.sort_values(['location', 'datetime'], inplace=True).reset_index(drop=True)

apply_corrections("WOR")

/Users/henrieloy/ideo-POC/process_data.py:244: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
/Users/henrieloy/ideo-POC/process_data.py:185: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


UnboundLocalError: cannot access local variable 'past_mask' where it is not associated with a value